# 分词

## 1.1最大正向匹配法

1、读取字典文件

In [1]:
with open('../data/字典.txt', encoding='utf-8') as f:
    txt = f.readlines()
    
print(txt[:4])
dictionary = [i.split()[0] for i in txt]
print(dictionary[:4])
print(len(dictionary))

['AT&T 3 nz\n', 'B超 3 n\n', 'c# 3 nz\n', 'C# 3 nz\n']
['AT&T', 'B超', 'c#', 'C#']
349046


2、最大正向匹配法

In [2]:
sentence = '今天我来到北京清华大学。'
max_len = 5

In [3]:
words = []
while len(sentence) != 0:
    tmp = sentence[:max_len]
    while tmp not in dictionary and len(tmp) != 1:
        tmp = tmp[: -1]
    words.append(tmp)
    sentence = sentence[len(tmp):]
    print(sentence)

print(words)

我来到北京清华大学。
来到北京清华大学。
北京清华大学。
清华大学。
。

['今天', '我', '来到', '北京', '清华大学', '。']


3、自定义函数

In [4]:
def fenci(sentence, dic, max_len=5):
    words = []
    while len(sentence) != 0:
        tmp = sentence[:max_len]
        while tmp not in dic and len(tmp) != 1:  # 要注意添加len(tmp) != 1这个添加，不然会陷入死循环，最后一个符号出不来
            tmp = tmp[: -1]      
        words.append(tmp)
        sentence = sentence[len(tmp):]
    print(words)
    
    return words

In [5]:
fenci(sentence='今天我们学习文本挖掘。', dic=dictionary, max_len=4)

['今天', '我们', '学习', '文本', '挖掘', '。']


['今天', '我们', '学习', '文本', '挖掘', '。']

## 1.2 最大逆向匹配法

读取字典

In [6]:
with open('../data/字典.txt', encoding='utf-8') as f:
    txt = f.readlines()
dictionary = [i.split()[0] for i in txt]

自定义实现

In [7]:
sentence = '今天我来到北京清华大学。'
max_len = 5

def fenci2(sentence, dic, max_len):
    words = []
    while(len(sentence) != 0):
        tmp = sentence[-max_len: ]
        while tmp not in dic and len(tmp) != 1:
            tmp = tmp[1:]
        words = [tmp] + words
        sentence = sentence[:-len(tmp)]
    
    return words

fenci2(sentence='今天我们学习文本挖掘。', dic=dictionary, max_len=4)

['今天', '我们', '学习', '文本', '挖掘', '。']

## 1.3 隐式马可夫模型HMM

数据导入

In [1]:
p_start = {'good':0.63, 'normal':0.17, 'bad':0.2}
p_emit = {
    'good':{'working':0.05, 'traval':0.35, 'shopping':0.35, 'running':0.25},
    'normal':{'working':0.25, 'traval':0.25, 'shopping':0.25, 'running':0.25},
    'bad':{'working':0.6, 'traval':0.2, 'shopping':0.05, 'running':0.15}
}
p_trans = {
    'good':{'good':0.5, 'normal':0.375, 'bad':0.125},
    'normal':{'good':0.25, 'normal':0.125, 'bad':0.625},
    'bad':{'good':0.25, 'normal':0.375, 'bad':0.375}
}
# 不容易构建、保存，但是索引比较容易

假设观察到K连续3天的行为分布是：工作、购物、旅行。那么K三天的心情是什么样子的？

-- 穷举法

In [3]:
obs = ['working', 'shopping', 'traval']
states = ['good', 'normal', 'bad']

V = [{}]  # 记录条路径及相应的概率

# 初始化
for y in states:
    V[0][y] = p_start[y] * p_emit[y][obs[0]]
print(V)


for t in range(1, len(obs)): 
    V.append({})
    for y in states:  # t时刻的心情状态
        for i, j in V[-2].items():
            pa = i.split('-')[-1]  # t-1时刻的心情状态
            V[t][i+'-'+y] = j * p_emit[y][obs[t]] * p_trans[pa][y]

print(V[-1])
(prob, path) = max((j, i) for i, j in V[-1].items())
print(f'连续状态为{obs}后，心情可能是：{path}')

[{'good': 0.0315, 'normal': 0.0425, 'bad': 0.12}]
{'good-good-good': 0.0009646874999999999, 'normal-good-good': 0.0006507812499999999, 'bad-good-good': 0.0018374999999999997, 'good-normal-good': 0.0002583984375, 'normal-normal-good': 0.0001162109375, 'bad-normal-good': 0.000984375, 'good-bad-good': 1.72265625e-05, 'normal-bad-good': 0.0001162109375, 'bad-bad-good': 0.000196875, 'good-good-normal': 0.000516796875, 'normal-good-normal': 0.0003486328125, 'bad-good-normal': 0.000984375, 'good-normal-normal': 9.228515625e-05, 'normal-normal-normal': 4.150390625e-05, 'bad-normal-normal': 0.0003515625, 'good-bad-normal': 1.845703125e-05, 'normal-bad-normal': 0.00012451171875, 'bad-bad-normal': 0.00021093750000000002, 'good-good-bad': 0.0001378125, 'normal-good-bad': 9.296875e-05, 'bad-good-bad': 0.0002625, 'good-normal-bad': 0.000369140625, 'normal-normal-bad': 0.000166015625, 'bad-normal-bad': 0.00140625, 'good-bad-bad': 1.4765625000000001e-05, 'normal-bad-bad': 9.960937500000001e-05, 'bad-b

In [10]:
print(tmp)
print(V)

[['good', 'normal', 'bad'], ['good', 'good', 'good', 'normal', 'normal', 'normal', 'bad', 'bad', 'bad'], ['good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'bad', 'bad', 'bad', 'bad', 'bad', 'bad', 'bad', 'bad', 'bad']]
[{'good': 0.0315, 'normal': 0.0425, 'bad': 0.12}, {'good-good': 0.0055125, 'normal-good': 0.00371875, 'bad-good': 0.010499999999999999, 'good-normal': 0.002953125, 'normal-normal': 0.001328125, 'bad-normal': 0.01125, 'good-bad': 0.000196875, 'normal-bad': 0.001328125, 'bad-bad': 0.0022500000000000003}, {'good-good-good': 0.0009646874999999999, 'normal-good-good': 0.0006507812499999999, 'bad-good-good': 0.0018374999999999997, 'good-normal-good': 0.0002583984375, 'normal-normal-good': 0.0001162109375, 'bad-normal-good': 0.000984375, 'good-bad-good': 1.72265625e-05, 'normal-bad-good': 0.0001162109375, 'bad-bad-good': 0.000196875, 'good-good-normal': 0.00051679687

-- 维特比算法

In [24]:
# obs = ['working', 'shopping', 'traval']
# states = ['good', 'normal', 'bad']

# V = [{}]
# path = {}

# # 初始化
# for y in states:
#     V[0][y] = p_start[y] * p_emit[y][obs[0]]
#     path[y] = [y]
# print(path)
# print(V)

# # 从第二天开始
# for t in range(1, 3):
#     V.append({})
#     newpath = {}
#     for y in states:
#         em_p = p_emit[y][obs[t]]
#         (prob, state) = max([(V[t - 1][y0] * p_trans[y0][y] * em_p, y0) for y0 in states])
#         V[t][y] = prob
#         newpath[y] = path[state] + [y]
#     print(newpath)
#     path = newpath

# (prob, state) = max((V[len(obs) - 1][y], y) for y in states)  # 确定三条路径的最优解

# print(f'最后一天的心情为：{state}, 概率为：{prob}')
# print(f'连续状态为{obs}后，心情可能是：{path[state]}')

In [11]:
obs = ['working', 'shopping', 'traval']
states = ['good', 'normal', 'bad']
V = [{}]
path = [{}]

# 初始化
for y in states:
    V[0][y] = p_start[y] * p_emit[y][obs[0]]
    path[0][y] = [y]
    
print('\n', V)
print('\n', path)

for t in range(1, len(obs)):
    V.append({})
    newpath = {}
    for y in states:
        em_p = p_emit[y][obs[t]]
        (prob, state) = max((V[-2][y0]*p_trans[y0][y]*em_p, y0) for y0 in states)
        V[-1][y] = prob
        newpath[y] = path[-1][state] + [y]
    path.append(newpath)
    
print('\n', V)
print('\n', path)

(prob, state) = max((V[-1][y], y) for y in states)  # 确定三条路径的最优解

print(f'\n最后一天的心情为：{state}, 概率为：{prob}')
print(f'\n连续状态为{obs}后，心情可能是：{path[-1][state]}')


 [{'good': 0.0315, 'normal': 0.0425, 'bad': 0.12}]

 [{'good': ['good'], 'normal': ['normal'], 'bad': ['bad']}]

 [{'good': 0.0315, 'normal': 0.0425, 'bad': 0.12}, {'good': 0.010499999999999999, 'normal': 0.01125, 'bad': 0.00225}, {'good': 0.0018374999999999997, 'normal': 0.000984375, 'bad': 0.00140625}]

 [{'good': ['good'], 'normal': ['normal'], 'bad': ['bad']}, {'good': ['bad', 'good'], 'normal': ['bad', 'normal'], 'bad': ['bad', 'bad']}, {'good': ['bad', 'good', 'good'], 'normal': ['bad', 'good', 'normal'], 'bad': ['bad', 'normal', 'bad']}]

最后一天的心情为：good, 概率为：0.0018374999999999997

连续状态为['working', 'shopping', 'traval']后，心情可能是：['bad', 'good', 'good']


In [12]:
V

[{'bad': 0.12, 'good': 0.0315, 'normal': 0.0425},
 {'bad': 0.00225, 'good': 0.010499999999999999, 'normal': 0.01125},
 {'bad': 0.00140625, 'good': 0.0018374999999999997, 'normal': 0.000984375}]

In [13]:
print(path)

[{'good': ['good'], 'normal': ['normal'], 'bad': ['bad']}, {'good': ['bad', 'good'], 'normal': ['bad', 'normal'], 'bad': ['bad', 'bad']}, {'good': ['bad', 'good', 'good'], 'normal': ['bad', 'good', 'normal'], 'bad': ['bad', 'normal', 'bad']}]


2、 HMM应用--分词  
jieba程序的路径：C:\Users\45543\AppData\Local\Continuum\Anaconda3\Lib\site-packages\jieba  
中文字符集合：[\u4E00-\u9FD5]

In [14]:
# 导入父级目录下的Python文件
import sys
sys.path.append(r'C:\Users\45543\Desktop\NLP\data')  # 这种方法属于一次性的，只对当前的python解释器进程有效，关掉python重启后就失效了
from hmm.prob_start import P as start_P
from hmm.prob_trans import P as trans_P
from hmm.prob_emit import P as emit_P

In [15]:
print(start_P)
print('\n', trans_P)

{'B': -0.26268660809250016, 'E': -3.14e+100, 'M': -3.14e+100, 'S': -1.4652633398537678}

 {'B': {'E': -0.51082562376599, 'M': -0.916290731874155}, 'E': {'B': -0.5897149736854513, 'S': -0.8085250474669937}, 'M': {'E': -0.33344856811948514, 'M': -1.2603623820268226}, 'S': {'B': -0.7211965654669841, 'S': -0.6658631448798212}}


In [16]:
obs = sentence = '今天我来到北京清华大学'
states = 'BEMS'

V = [{}]
path = {}

PrevStatus = {
    'B': 'ES',
    'M': 'MB',
    'S': 'SE',
    'E': 'BM'   # t时刻状态:t-1时刻状态
}

for y in states:
    V[0][y] = start_P[y] + emit_P[y][obs[0]]
    path[y] = [y]
print(V, path)

for t in range(1, len(obs)):
    V.append({})
    newpath = {}
    for y in states:
        em_p = emit_P[y][obs[t]]
        
        (prob, state) = max((em_p + V[t-1][y0] + trans_P[y0][y], y0) for y0 in PrevStatus[y])  # y0是上一个状态
        V[t][y] = prob
        newpath[y] = path[state] + [y]
    path = newpath

(prob, state) = max((V[len(obs) - 1][y], y) for y in 'ES')  # 注意'ES'，句子最后一个状态只能是这两种
print(prob, state)
print(path)
pos_list = path[state]

res = []
for i, char in enumerate(obs):    
    sign = pos_list[i]
    if sign == 'B':
        begin = i
    elif sign == 'E':
        res.append(obs[begin: i+1])
    elif sign == 'S':
        res.append(obs[i])
    
print(f'''
{obs}  
标注结果为：
  {path[state]}  
分词结果为：
  {res}
''')

[{'B': -6.8985414338214115, 'E': -3.14e+100, 'M': -3.14e+100, 'S': -8.598245859008907}] {'B': ['B'], 'E': ['E'], 'M': ['M'], 'S': ['S']}
-68.04528930719714 E
{'B': ['B', 'E', 'S', 'S', 'S', 'B', 'E', 'B', 'M', 'E', 'B'], 'E': ['B', 'E', 'S', 'S', 'S', 'B', 'E', 'B', 'M', 'M', 'E'], 'M': ['B', 'E', 'S', 'S', 'S', 'B', 'E', 'B', 'M', 'M', 'M'], 'S': ['B', 'E', 'S', 'S', 'S', 'B', 'E', 'B', 'M', 'E', 'S']}

今天我来到北京清华大学  
标注结果为：
  ['B', 'E', 'S', 'S', 'S', 'B', 'E', 'B', 'M', 'M', 'E']  
分词结果为：
  ['今天', '我', '来', '到', '北京', '清华大学']



In [ ]:
pat = path[state]
res = []
for i, (word, char) in enumerate(zip(obs, pat)):
    if char == 'S':
        res.append(word)
    elif char == 'B':
        begin = i
    elif char == 'E':
        res.append(obs[begin:(i+1)])
print(res)

In [17]:
def viterbi(obs, states, start_P, trans_P, emit_P):
    V = [{}]
    path = {}

    PrevStatus = {
        'B': 'ES',
        'M': 'MB',
        'S': 'SE',
        'E': 'BM'   # t时刻状态:t-1时刻状态
    }

    for y in states:
        V[0][y] = start_P[y] + emit_P[y][obs[0]]
        path[y] = [y]
    print(V, path)

    for t in range(1, len(obs)):
        V.append({})
        newpath = {}
        for y in states:
            em_p = emit_P[y][obs[t]]

            (prob, state) = max((em_p + V[t-1][y0] + trans_P[y0][y], y0) for y0 in PrevStatus[y])  # y0是上一个状态
            V[t][y] = prob
            newpath[y] = path[state] + [y]
        path = newpath
    (prob, state) = max((V[len(obs) - 1][y], y) for y in 'ES')
    
    return (prob, path[state])

In [18]:
viterbi(sentence, 'BMES', start_P, trans_P, emit_P)  # 调用自定义函数

[{'B': -6.8985414338214115, 'M': -3.14e+100, 'E': -3.14e+100, 'S': -8.598245859008907}] {'B': ['B'], 'M': ['M'], 'E': ['E'], 'S': ['S']}


(-68.04528930719714, ['B', 'E', 'S', 'S', 'S', 'B', 'E', 'B', 'M', 'M', 'E'])

In [19]:
print(f'step{2:>6}')
print(f'step{200:>6}')

step     2
step   200


## 1.4 jieba分词

In [20]:
import jieba
print(jieba.lcut('今天我来到北京清华大学'))
print(jieba.lcut_for_search('今天我来到北京清华大学'))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\45543\AppData\Local\Temp\jieba.cache
Loading model cost 0.758 seconds.
Prefix dict has been built succesfully.


['今天', '我', '来到', '北京', '清华大学']
['今天', '我', '来到', '北京', '清华', '华大', '大学', '清华大学']


对《鹿鼎记》进行分词

In [21]:
with open('../data/鹿鼎记.txt', 'r', encoding='utf-8') as f:
    txt = f.read()
    
txt = txt.split()
print(txt[:7])

['\ufeff', '《鹿鼎记》', '作者：金庸', '正文', '第一回', '纵横钩党清流祸', '峭茜风期月旦评']


In [22]:
import jieba

# 导入自定义字典
jieba.load_userdict('../data/coal_dict.txt')

words = [jieba.lcut(t) for t in txt]

from tkinter import _flatten
words = list(_flatten(words))

In [23]:
words[:23]

['\ufeff',
 '《',
 '鹿鼎记',
 '》',
 '作者',
 '：',
 '金庸',
 '正文',
 '第一回',
 '纵横',
 '钩',
 '党',
 '清流',
 '祸',
 '峭茜',
 '风期',
 '月旦评',
 '北风',
 '如刀',
 '，',
 '满地',
 '冰霜',
 '。']